# New issues in the news

The goal of this tutorial is to demonstrate the LSEG Data Library for Python with the focus on the news retrieval in a Jupyter Notebook environment. So, for that purpose we are going to look at new issue news from International Financial Review (IFR), a global capital markets intelligence provider, that is a part of LSEG.

We will capture the `PRICED` or `DEAL` notifications that contain structured text that we will extract.

Before we start, let's make sure that:

+ LSEG Workspace desktop application is up and running;
+ LSEG Data Library for Python is installed;
+ You have created an application ID for this script.

If you have not yet done this, have a look at the quick start section for this API. 

*A general note on the Jupyter Notebook usage*: in order to execute the code in the cell, press <kbd>Shift</kbd>+<kbd>Enter</kbd>. While notebook is busy running your code, the cell will look like this: `In [*]`. When its finished, you will see it change to the sequence number of the task and the output, if any.

For more info on the Jupyter Notebook, check out Project Jupyter site http://jupyter.org or 'How to set up a Python development environment for LSEG Workspace' tutorial on this portal.

Let's start with referencing LSEG Data Library for Python and pandas:

In [1]:
import lseg.data as ld
import pandas as pd
import os

app_key = os.getenv("APP_KEY")


Paste your application ID in to this line:

In [2]:
ld.open_session(app_key=app_key)

<lseg.data.session.Definition object at 0x21727d4c140 {name='workspace'}>

We are going to request emerging market new issue (**ISU**) eurobond (**EUB**) news from International Financial Review Emerging EMEA service (**IFREM**), focusing on the notifications of the already priced issues. You can replicate this request in the **News Monitor** app with the following query:

+ `Product:IFREM AND Topic:ISU AND Topic:EUB AND ("PRICED" OR "DEAL")`

In [5]:
from datetime import date

start_date, end_date = date(2024, 1, 1), date.today()
q = 'Product:IFREM AND Topic:ISU AND Topic:EUB AND (\"PRICED:\" OR \"DEAL:\")'
headlines = ld.news.get_headlines(q,start=start_date,end=end_date, count = 100)
headlines.head()

,headline,storyId,sourceCode
versionCreated,,,
2025-04-24 13:46:03.012,PRICED: LHV Group €50m PNC5 AT1,urn:newsml:reuters.com:20250424:nIfp7mMCSV:1,NS:IFR
2025-04-24 12:03:55.000,UPDATE PRICED: OCP $750m 5yr & $1bn long 10yr,urn:newsml:reuters.com:20250424:nIfp6SWcZk:1,NS:IFR
2025-04-24 12:03:55.000,UPDATE PRICED: OCP $750m 5yr & $1bn long 10yr,urn:newsml:reuters.com:20250424:nIfp4fJ3xZ:1,NS:IFR
2025-04-24 07:10:00.000,UPDATE PRICED: OCP $750m 5yr & $1bn long 10yr,urn:newsml:reuters.com:20250424:nIfp401Bg9:1,NS:IFR
2025-04-24 07:10:00.000,UPDATE PRICED: OCP $750m 5yr & $1bn long 10yr,urn:newsml:reuters.com:20250424:nIfp2kcCRd:1,NS:IFR


In the context of news, each story has its own unique idenifier, created according to the RFC 3085 standard. Here's what the story looks like, notice that I am using the standard HTML() function from Notebook to display it:

In [6]:
from IPython.core.display import HTML
html_doc = ld.news.get_story(headlines['storyId'][1])
HTML(html_doc)

C:\Users\U8009686\AppData\Local\Temp\ipykernel_28536\2221378203.py:2:FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


Issuer Long Name:,OCP SA,Country:,MOROCCO
Issuer Short Name:,OCP,Country of Risk:,MOROCCO
LEI:,213800D26TAPVTCVWG40,,
Moodys:,Baa3,S&P:,BB+
Fitch:,BB+,,
,,Issue Type:,bmk / snr unsec
,,Format:,RegS/144A
Currency Size:,USD750m,Pricing Date:,2025-04-23
Settlement Date:,2025-04-30,Maturity (Tenor):,2030-04-30 (5.00yr)
,,Coupon/Coupon Type:,6.1 Fixed
,,Reoffer Price:,99.006


In [7]:
from bs4 import BeautifulSoup

def termsheet_to_dict(html_doc):
    clean_matches = {}
    soup = BeautifulSoup(html_doc, 'html.parser')
    for tr in soup.find_all('tr'):
        all_td =tr.find_all('td')
        key = all_td[0].get_text().split(':')[0]
        value = all_td[1].get_text()
        if key != '\xa0':
            clean_matches[key]=value
        if(len(all_td)>2):
            key = all_td[2].get_text().split(':')[0]
            value = all_td[3].get_text()
            if key != '\xa0':
                clean_matches[key]=value
    return clean_matches

Now we can parse the data using the BeautifulSoup module. Let's create a function that is going to return a dictionary from the this type of article.

Let's test it and see if it works:

In [8]:
termsheet_to_dict(html_doc)['Notes']

'Adds total books. USD April 2030 & Mar 2036 snr unsec bmk RegS/144A. Baa3/BB+/BB+. Gloco/Active: Citi(log)/JPM(B&D), passive: BNPP/DB/Miz/SMBC. IPTs 5yr T+265 area, L10yr T+290 area. Launch 750m 5yr at T+235, 1bn 10yr at T+260. UST 4.000% due Mar-2030 @ 100-02 / 3.985% / HR 95%, UST 4.625% due Feb-2035 @ 102-02 / 4.364% / HR 93%. Combined books 7bn.\n'

Let's extract all data for all headlines:

In [9]:
from time import sleep

result = []

index = pd.DataFrame(headlines, columns=['storyId']).values.tolist()

for i, storyId in enumerate(index):
    story = ld.news.get_story(storyId[0])
    x = termsheet_to_dict(story)
    if x:
        result.append(x)
    sleep(0.5)

df = pd.DataFrame(result)
df.head()

,Issuer Long Name,Country,Issuer Short Name,Country of Risk,LEI,Moodys,Issue Type,Format,Currency Size,Pricing Date,...,Make Whole Call,Coc Call,Guarantor,PP,XD,Price,Tap Size,Equity Claw Back,Links,Fees
0,LHV Group AS,ESTONIA,LHV Group,ESTONIA,529900JG015JC10LED24,Ba3,Tier 1,RegS,EUR50m,2025-04-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OCP SA,MOROCCO,OCP,MOROCCO,213800D26TAPVTCVWG40,Baa3,bmk / snr unsec,RegS/144A,USD750m,2025-04-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OCP SA,MOROCCO,OCP,MOROCCO,213800D26TAPVTCVWG40,Baa3,bmk / snr unsec,RegS/144A,USD1bn,2025-04-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OCP SA,MOROCCO,OCP,MOROCCO,213800D26TAPVTCVWG40,Baa3,bmk / snr unsec,RegS/144A,USD1bn,2025-04-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OCP SA,MOROCCO,OCP,MOROCCO,213800D26TAPVTCVWG40,Baa3,bmk / snr unsec,RegS/144A,USD750m,2025-04-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, when we have the dataframe in place, we can perform simple stats on our data. For instance:

In [10]:
df['Issue Type'].value_counts()

Issue Type
bmk /  snr unsec            38
bmk / snr unsec             12
bmk                         11
 snr unsec                   9
bmk /  snr unsec / sukuk     8
bmk / Tier 2                 4
 snr unsec / sukuk           3
Tier 1                       2
hybrid / bmk                 2
bmk / sukuk                  2
bmk / Tier 1                 1
bmk / snr sec                1
covered                      1
bmk / covered                1
snr unsec                    1
bmk / snr unsec / sukuk      1
bmk / sukuk / Tier 1         1
Name: count, dtype: int64

What about a specific country?

In [11]:
df[df['Country']=='MOROCCO']['Issue Type'].value_counts()

Issue Type
bmk /  snr unsec    6
bmk                 4
 snr unsec          2
Name: count, dtype: int64

You can experiment further by changing the original headline search query, for example, by including the RIC into your request.